In [1]:
# Import libraries
from torch import nn,cuda,backends, optim, save, load, no_grad
import torch
from torchvision.datasets import ImageFolder, CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import time
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import pickle 
from pathlib import Path

In [2]:
mean=[0.4914, 0.4822, 0.4465]
std=[0.2023, 0.1994, 0.2010]
transformations=transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomInvert(p=0.2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=mean,std=std),
    ])

In [3]:
batch_size=50
classes = torch.tensor([3, 8])
classes_names= ["cat","ship"]

data_train = CIFAR10(root='./data', train=True, download=True, transform=transformations)
indices = (torch.tensor(data_train.targets)[..., None] == classes).any(-1).nonzero(as_tuple=True)[0]
data_train = torch.utils.data.Subset(data_train, indices)
dataloader_train = torch.utils.data.DataLoader(data_train,batch_size=batch_size, shuffle=True, num_workers=2)

data_test = CIFAR10(root='./data', train=False, download=True, transform=transformations)
indices = (torch.tensor(data_test.targets)[..., None] == classes).any(-1).nonzero(as_tuple=True)[0]
data_test = torch.utils.data.Subset(data_test, indices)
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = (
    "cuda"
    if cuda.is_available()
    else "mps"
    if backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
class NN(nn.Module):
    def __init__(self, input_layer=3*32*32, output_layer=10):
        super().__init__()
        # Define layers
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_layer,678),
            nn.ReLU(),
            nn.Linear(678,output_layer),
            nn.Softmax(dim=1),
        )

    def forward(self, x):        
        x=self.linear_relu_stack(x)   
        return x

In [6]:
# Funtion to train model
def train_model(model, dataloader, checkpoint_data=None,criterion=None,optimizer=None, continue_from_epoch=0, epoches=5, limit_batches=-1,dataloader_test=None,path=""):
    if (criterion is None):
        criterion=nn.CrossEntropyLoss()
    if (optimizer is None):
        optimizer=optim.Adam(model.parameters(),lr=0.001)
    if (checkpoint_data is None):
        time_model=0
        best_epoch=(-1,999999999)
    else:
        time_model=checkpoint_data["time"]
        best_epoch=checkpoint_data["best_epoch"]
    f=open(path+"/model_stats.txt","w")
    max_test_acc=0
    counter=0

    for epoch in range(continue_from_epoch,continue_from_epoch+epoches):
        subloss=0
        total_loss=0
        start_time_epoch=time.time()
        start_time = time.time()
        for batch_number,(x_train,y_train) in enumerate(dataloader):
            x_train,y_train=x_train.to(device),y_train.to(device)
            y_pred=model(x_train)
            #print(y_pred.shape)
            #print(y_train.shape)            
            #print(y_pred[0])
            #print(y_train[0])
            loss=criterion(y_pred,y_train)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        stop_time=time.time()
        time_model=time_model+(stop_time-start_time_epoch)
        if (total_loss<best_epoch[1]):
            best_epoch=(epoch,total_loss)
        if (dataloader_test is not None):
            print ("Testing...")
            Accuracy_train=test_model(model,dataloader)
            start_testing_time=time.time()
            Accuracy_test=test_model(model,dataloader_test)            
            stop_testing_time=time.time()
            print(f"Epoch {epoch} Loss: {total_loss}, Time: {stop_time-start_time_epoch} seconds, Accuracy on Test: {Accuracy_test}, Accuracy on Train: {Accuracy_train}, Training time:{time_model}, Testing time:{stop_testing_time-start_testing_time}")
        if (abs(Accuracy_test-Accuracy_train)>0.1):
            print("Model has oveefitted. Stopping training...")
            print("Model has oveefitted. Stopping training...", file=f)
            break
        if (max_test_acc>= Accuracy_test):
            counter+=1
        else:
            max_test_acc=Accuracy_test
            counter=0
        if (counter>=50):
            break
    f.close()
    return best_epoch

# Function to test model
def test_model(model,dataloader,limit_batches=-1):
    samples_tested=0
    correct=0
    with no_grad():
        for batch_number,(x_test,y_test) in enumerate(dataloader):
            x_test,y_test=x_test.to(device),y_test.to(device)
            y_pred=model(x_test)


            samples_tested+=y_test.size(0)
            prediction=torch.max(y_pred,1)[1]
            correct+=(prediction==y_test).sum()
            #if (batch_number%100==0):
            #    print(f"Batch number: {batch_number}, Accuraccy: {correct/samples_tested}")
            #if (limit_batches==batch_number):
            #    break
    accuracy= correct/samples_tested
    return (accuracy)

In [7]:
#model=NeuralNetwork(output_features=1,dropout_percent=0)
model=NN()
model.to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001)
train_model(model,dataloader_train, optimizer=optimizer,criterion=nn.MultiMarginLoss(), continue_from_epoch=0, epoches=8000, dataloader_test=dataloader_test,limit_batches=-1,path="/home/i/ioakeime/Models/svms/")


Testing...
Epoch 0 Loss: 61.73508155345917, Time: 7.151276588439941 seconds, Accuracy on Test: 0.7345000505447388, Accuracy on Train: 0.7299999594688416, Training time:7.151276588439941, Testing time:0.4047207832336426
Testing...
Epoch 1 Loss: 52.48224910348654, Time: 1.7651491165161133 seconds, Accuracy on Test: 0.7330000400543213, Accuracy on Train: 0.7175999879837036, Training time:8.916425704956055, Testing time:0.41390347480773926
Testing...
Epoch 2 Loss: 53.06178364157677, Time: 1.7735636234283447 seconds, Accuracy on Test: 0.737000048160553, Accuracy on Train: 0.7421999573707581, Training time:10.6899893283844, Testing time:0.40169596672058105
Testing...
Epoch 3 Loss: 51.41050901263952, Time: 1.7563090324401855 seconds, Accuracy on Test: 0.7550000548362732, Accuracy on Train: 0.7319999933242798, Training time:12.446298360824585, Testing time:0.42014431953430176
Testing...
Epoch 4 Loss: 53.9172126352787, Time: 1.7943685054779053 seconds, Accuracy on Test: 0.7610000371932983, Accu

(91, 44.79017509520054)